In [1]:
from pytfa.io.json import load_json_model
model = load_json_model('./../models/TFA_GEM_Recon3_Lehninger_Curated_prepared.json')

2025-01-30 12:05:42,861 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# This is the models with the original annotation 
# TODO: Put the original annotation back into the TFA 
# (its lost becuse the TFA model was made in matTFA ... )
from cobra.io.json import load_json_model
recon3 = load_json_model('./../models/Recon3D.json')

In [3]:
# Add a defined medium 

medium_metabolites = {
    'EX_o2_e': -100,
    'EX_h2o_e': -100,
    'EX_pi_e': -100,
    'EX_nh4_e': -100,
    'EX_h_e': -100,
    'EX_so4_e': -100,
    'EX_hco3_e': -100,
    'EX_h2o2_e': -100,
    'EX_glc_D_e': -100,
    'EX_ac_e': -100,
    'EX_lac_L_e': -100,
    'EX_hdca_e': -100,
    'EX_lnlc_e': -100,
    'EX_ocdcea_e': -100,
    'EX_ala_L_e': -100,
    'EX_arg_L_e': -100,
    'EX_asn_L_e': -100,
    'EX_asp_L_e': -100,
    'EX_cys_L_e': -100,
    'EX_gln_L_e': -100,
    'EX_glu_L_e': -100,
    'EX_gly_e': -100,
    'EX_his_L_e': -100,
    'EX_ile_L_e': -100,
    'EX_leu_L_e': -100,
    'EX_lys_L_e': -100,
    'EX_met_L_e': -100,
    'EX_phe_L_e': -100,
    'EX_pro_L_e': -100,
    'EX_ser_L_e': -100,
    'EX_thr_L_e': -100,
    'EX_trp_L_e': -100,
    'EX_tyr_L_e': -100,
    'EX_val_L_e': -100,
    'EX_bhb_e': -100,
    'EX_acac_e': -100,
}

for m, b in medium_metabolites.items():
    model.reactions.get_by_id(m).lower_bound = b

# Also allow for cytostolic ATP to be produced
model.reactions.cyt_atp2adp.lower_bound = -100

In [4]:
# Read the met_label table 
import pandas as pd

met_labels = pd.read_excel('Met_labeling_metabolites.xlsx')

In [5]:
from metabolink.annotation import find_metabolites_from_annotation




def find_metabolites_from_annotation(id, model, id_type='kegg.compound'):
    """
    Find metabolites in a model based on a given annotation id

    Parameters
    ----------
    id : str
        The annotation id to search for
    model : cobra.Model
        The model to search in  
        
    """

    # Veryfi if id is a non empty string
    if id and isinstance(id, str):
        metabolites = [] 
        for met in model.metabolites:
            try:
                ids = met.annotation[id_type]
            except KeyError:
                continue

            # TODO fix HMDB old vs new
            if id_type == 'hmdb':
                # Compare only last 5 digits of HMDB ids
                if id[-5:] in [i[-5:] for i in ids]:
                    metabolites.append(met.id)

            # For all other id_types compare the full id
            else:
                if id in ids:
                    metabolites.append(met.id)

        if metabolites not in [[], None]:            
                return metabolites
        else:
                return []
    else:
        return []
    

# Try to dinf the metablie labels using KEGG and HMDB 
kegg_2_met = met_labels.apply(lambda x: find_metabolites_from_annotation(x['KEGG'], recon3, id_type='kegg.compound'), axis=1)
hmdb_2_met = met_labels.apply(lambda x: find_metabolites_from_annotation(x['HMDB'], recon3, id_type='hmdb'), axis=1)


In [6]:
metabolites = pd.concat([kegg_2_met , hmdb_2_met], axis=1, keys=['KEGG', 'HMDB'])

# Print the counts of non nan values  
print('KEGG:', kegg_2_met.count())
print('HMDB:', hmdb_2_met.count())

# Joun Kegg and HMDB columns using set logic
metabolites['metabolites'] = metabolites.apply(lambda x: set(x['KEGG']) | set(x['HMDB']) , axis=1)

KEGG: 23
HMDB: 23


In [7]:
met_labels['model_metabolites'] = metabolites['metabolites']

In [8]:
# Check only for the inputs 
defined_precursors = [ 'EX_glc_D_e', 'EX_ac_e', 'EX_lac_L_e', 'EX_hdca_e', 'EX_lnlc_e', 'EX_ocdcea_e', 
                      'EX_ala_L_e', 'EX_arg_L_e', 'EX_asn_L_e', 'EX_asp_L_e', 'EX_cys_L_e', 'EX_gln_L_e', 'EX_glu_L_e',
                      'EX_gly_e', 'EX_his_L_e', 'EX_ile_L_e', 'EX_leu_L_e', 'EX_lys_L_e', 'EX_met_L_e', 'EX_phe_L_e',
                     'EX_pro_L_e', 'EX_ser_L_e', 'EX_thr_L_e', 'EX_trp_L_e', 'EX_tyr_L_e', 'EX_val_L_e', 'EX_bhb_e', 'EX_acac_e']

In [9]:
from metabolink import find_precursor_sets

# find precursor sets for all metabolites found in RECON3
precursor_sets = {}
for i, row in met_labels.iterrows():
    # Check if the metabolite was found in the model
    if row['model_metabolites'] != set():
        try:
            # Find the metabolite sets accros compartments // bases on the identifiers
            metabolites = list(row['model_metabolites'])
            
            # Drop metablolites that are in the extracellular environment
            metabolites = [m for m in metabolites if not m.endswith('_e')]

            # Replace dobuble underscore with single underscore (map to TFA recon3 annotation)
            metabolites = [m.replace('__', '_') for m in metabolites]
            
            precursor_set = find_precursor_sets(metabolites, model, defined_precursors=defined_precursors, method='min')
            precursor_sets[i] = precursor_set
        except ValueError as e:
            print(f'Error for {metabolites}: {e}')


Metabolites ['3sala_c', '3sala_m'] can be simultanously produced 74.9


2025-01-30 12:08:29,606 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Timeout limit is 3600s


2025-01-30 12:10:07,083 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


...

2025-01-30 14:38:23,703 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2025-01-30 14:38:42,938 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-01-30 14:38:42,939 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-01-30 14:38:42,940 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-01-30 14:38:42,942 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-01-30 14:38:42,944 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-01-30 14:38:42,945 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-01-30 14:38:42,946 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-01-30 14:38:42,948 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-01-30 14:38:42,979 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2025-01-30 14:38:42,981 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=ahcys_r: 100%|██████████| 3/3 [02:00<00:00, 40.32s/it]


In [10]:
from metabolink.io import extract_precursor_sets
table = extract_precursor_sets(precursor_sets)

/mnt/c/Users/dw6835/src/metabolink/metabolink/io.py:26: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  df.loc[k, list(p)] = True


ValueError: cannot reindex on an axis with duplicate labels

In [11]:
precursor_sets

{1: {'3sala_c': [defaultdict(int, {'cys_L_e': 1.4}),
   defaultdict(int, {'met_L_e': 83.961769})],
  '3sala_m': [defaultdict(int, {'cys_L_e': 100.0001}),
   defaultdict(int, {'met_L_e': 66.666733})]},
 2: {'5mta_c': [defaultdict(int, {'met_L_e': 15.060606})]},
 3: {'ach_c': [defaultdict(int, {'ser_L_e': 100.0001}),
   defaultdict(int, {'asn_L_e': 15.384615}),
   defaultdict(int, {'gln_L_e': 100.0001}),
   defaultdict(int, {'his_L_e': 100.0001}),
   defaultdict(int, {'glc_D_e': 4.0}),
   defaultdict(int, {'met_L_e': 30.638889}),
   defaultdict(int, {'thr_L_e': 100.0001}),
   defaultdict(int, {'gly_e': 100.0001}),
   defaultdict(int, {'lac_L_e': 26.0}),
   defaultdict(int, {'leu_L_e': 5.0}),
   defaultdict(int, {'pro_L_e': 100.0001}),
   defaultdict(int, {'cys_L_e': 7.05}),
   defaultdict(int, {'glu_L_e': 100.0001}),
   defaultdict(int, {'tyr_L_e': 75.0}),
   defaultdict(int, {'ile_L_e': 6.0}),
   defaultdict(int, {'lys_L_e': 100.0001}),
   defaultdict(int, {'ocdcea_e': 1.550186}),
   de

In [12]:
met_labels

,compound,formula,m/z,Met,validate,HMDB,KEGG,model_metabolites
0,3-Methylhistidine,C7H11N3O2,168.08,1,'5',HMDB0000479,C01152,{}
1,3-Sulfinoalanine,C3H7NO4S,152.00,3,'0',HMDB0000996,C00606,"{3sala_c, 3sala_m}"
2,5'-Methylthioadenosine,C11H15N5O3S,296.08,1,'1',HMDB0001173,C00170,"{5mta_c, 5mta_e}"
3,Acetylcholine,C7H15NO2,144.10,"1,2,3",'0',HMDB0000895,C01996,"{ach_e, ach_c, ach_n}"
4,Anserine,C10H16N4O3,239.11,1,'0',HMDB0000194,C01262,{}
5,Asymmetric dimethylarginine,C8H18N4O2,201.14,"1,2",'0',HMDB0001539,C03626,{}
6,Creatine,C4H9N3O2,130.06,1,'1',HMDB0000064,C00300,{}
7,Creatinine,C4H7N3O,112.05,1,'0',HMDB0000562,C00791,"{creat_e, creat_c, creat_m}"
8,Glutarylcarnitine,C12H21NO6,274.13,"1,2,3",'0',HMDB0013130,NaN,{}
9,Glutathione,C10H17N3O6S,306.08,3,'10',HMDB0000125,C00051,"{gthrd_c, gthrd_m, gthrd_e, gthrd_r}"
